In [ ]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = '62f6dc1bcc93404790b3901f2a437de0'
CLIENT_SECRET = '23bb2fb4bb65453cb5bf035c0f260041'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [ ]:
!pip install spotipy

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [ ]:
playlist_id = '3VXYosRrs1Bs0x3SgDB8dg'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                              Track Name  \
0                            Nuvve Nuvve   
1                       Em Sandeham Ledu   
2                   Manasuna Edho Raagam   
3              O Manasa - Female Version   
4                               Aanandam   
..                                   ...   
95                       Chitti Nadumune   
96                               Devatha   
97  Unnatundi Gundey - From "Ninnu Kori"   
98                           Evare Nuvvu   
99                             Yemantave   

                                              Artists  \
0                             Jonita Gandhi, Thaman S   
1                              Kalyani Malik, Sunitha   
2                            Harris Jayaraj, Chinmayi   
3                                      Shreya Ghoshal   
4   Anirudh Ravichander, Sathyaprakash, Shashaa Ti...   
..                                                ...   
95                         Mallikarjun, Premgi Amaren   
96             

In [ ]:
music_df.head()

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Nuvve Nuvve,"Jonita Gandhi, Thaman S",Kick 2,281SmfhMWT4OPQvlUthafC,1xbZ2No4vTyvc5sVcW4T9Q,50,2015-05-08,253231,False,https://open.spotify.com/track/1xbZ2No4vTyvc5s...,...,0.459,1,-12.298,0,0.0703,0.864,0.196000,0.142,0.328,168.008
1,Em Sandeham Ledu,"Kalyani Malik, Sunitha",Oohalu Gusagusalade (Original Motion Picture S...,5HaFZv77oP0GMVZXSfyz4h,6vsPBbLGfeLraBanG4HJaR,59,2014-04-28,232914,False,https://open.spotify.com/track/6vsPBbLGfeLraBa...,...,0.499,2,-5.958,1,0.0320,0.630,0.000002,0.112,0.541,127.915
2,Manasuna Edho Raagam,"Harris Jayaraj, Chinmayi",Yentha Vaadu Gaanie (Original Motion Picture S...,0cIEwBbPTnrICf9ueeClmR,5RJLNH6xaT1BI7xYtC4RQQ,47,2015-04-30,234573,False,https://open.spotify.com/track/5RJLNH6xaT1BI7x...,...,0.539,4,-6.203,1,0.0442,0.886,0.000000,0.105,0.249,131.920
3,O Manasa - Female Version,Shreya Ghoshal,Oka Manasu (Original Motion Picture Soundtrack),7G5b4IGgR2Apv54SLzZghF,34FyP3mlIosuIfd65Cr0l7,40,2016-05-18,294000,False,https://open.spotify.com/track/34FyP3mlIosuIfd...,...,0.395,0,-7.411,1,0.0305,0.557,0.000021,0.107,0.284,119.825
4,Aanandam,"Anirudh Ravichander, Sathyaprakash, Shashaa Ti...",Vivekam (Original Motion Picture Soundtrack),0Am0zjniB5sapTdgBeYl9m,0aZ4UJvNxXrtIKpUk2wtdo,30,2017-08-16,256470,False,https://open.spotify.com/track/0aZ4UJvNxXrtIKp...,...,0.686,4,-8.825,1,0.0379,0.640,0.000508,0.100,0.555,114.785


In [ ]:
print(len(music_df))

100


In [ ]:
print(music_df.columns)


Index(['Track Name', 'Artists', 'Album Name', 'Album ID', 'Track ID',
       'Popularity', 'Release Date', 'Duration (ms)', 'Explicit',
       'External URLs', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode',
       'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo'],
      dtype='object')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [ ]:
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    #content_based_recommendations = music_df.iloc[similar_song_indices]
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity','External URLs']]

    return content_based_recommendations

In [ ]:
input_song_name = "Nuvve Nuvve"
recommendations = content_based_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations.head()

Hybrid recommended songs for 'Nuvve Nuvve':


,Track Name,Artists,Album Name,Release Date,Popularity,External URLs
86,Priyathama Priyathama,Chinmayi,Majili,2019-03-11,55,https://open.spotify.com/track/4wHZiaLsuI9aQdW...
8,Paravaledu Paravaledu,Geetha Madhuri,Manasara,2013-01-15,41,https://open.spotify.com/track/3YqxMRlG0Vccmb0...
48,Jala Jala Jalapaatham Nuvvu,"Devi Sri Prasad, Jaspreet Jasz, Shreya Ghoshal",Uppena,2021-01-31,50,https://open.spotify.com/track/7HhiFHVmeM50rHX...
43,Daredumdadum,"Mickey J. Meyer, Sai Shivani",Mukunda,2014-12-03,48,https://open.spotify.com/track/42fjfbKn27YJ3ho...
45,Chinukai Kurisindhi,Shahid Mallya,Paper Boy,2018-08-31,28,https://open.spotify.com/track/5B5Lr5ZeEyUAM9E...
